# 7.1 CNN 개념

http://cs231n.github.io/convolutional-networks/

[cs231n 정리한 블로그](http://umbum.tistory.com/223)

[컨볼루셔널 뉴럴넷 (Convolutional Neural Network) - Terry님 블로그](http://t-robotics.blogspot.kr/2016/05/convolutional-neural-network_31.html#.WuXrcYiFNaQ)

[딥러닝 - 초보자를 위한 컨볼루셔널 네트워크를 이용한 이미지 인식의 이해](http://bcho.tistory.com/1149)

![https://1.bp.blogspot.com/-XJ5K5HJtK-I/V04GqJaBlNI/AAAAAAAAyt0/vFxfX3F-A4QqcMrJYxWot_dB0TsnL5wAwCLcB/s1600/Convolution_schematic.gif](https://1.bp.blogspot.com/-XJ5K5HJtK-I/V04GqJaBlNI/AAAAAAAAyt0/vFxfX3F-A4QqcMrJYxWot_dB0TsnL5wAwCLcB/s1600/Convolution_schematic.gif)

+ 컨볼루션: 윈도우에 해당하는 원소들끼리 각각 곱해서 더해주는 것

![https://2.bp.blogspot.com/-jnY4Ot3XDpY/V04LuiulEzI/AAAAAAAAyug/gd3tJLWj3Aw9H56Gz7opkTBxJrXbkM5RgCKgB/s1600/Screen-Shot-2015-11-05-at-2.18.38-PM.png](https://2.bp.blogspot.com/-jnY4Ot3XDpY/V04LuiulEzI/AAAAAAAAyug/gd3tJLWj3Aw9H56Gz7opkTBxJrXbkM5RgCKgB/s1600/Screen-Shot-2015-11-05-at-2.18.38-PM.png)

+ 풀링 : 사이즈를 줄이는 과정, 골라내기

## MLP(MultiLayer Perceptron) vs CNN

### MLP

(i) input data 입력

(ii) y=Wx+b 계산

(iii) activation function 적용

+ CNN은 (ii)에서 MLP와 차이

+ input data를 벡터화하여 input layer에 넣는 대신 2D 구조를 그대로 유지

### CNN

(ii-a) 컨볼루션을 적용해 새로운 2D 데이터를 얻고,

(ii-b) 여기에 activation function을 적용한 후

(ii-c) 풀링

+ 여기까지가 하나의 컨볼루션 레이어

(iii) Feed-Foward Neural Network를 하나 쌓아서 완성시킴

# 7.2 모델 구현하기

+ MNIST 데이터를 CNN으로 학습

In [2]:
# 이미지 처리 분야에서 가장 유명한 신경망 모델인 CNN 을 이용하여 더 높은 인식률을 만들어봅니다.
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


## 신경망 모델 구성

In [24]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32) # 드롭아웃을 위한 플레이스 홀더

In [25]:
print(X)

Tensor("Placeholder_6:0", shape=(?, 28, 28, 1), dtype=float32)


+ 기존 모델에서는 입력 값을 28x28 하나의 차원으로 구성하였으나,

+ CNN 모델을 사용하기 위해 2차원 평면과 특성치의 형태를 갖는 구조로 만듭니다.

+ 마지막 차원인 1은 특징의 개수로 MNIST 데이터는 회색조 이미지라 채널에 색상이 한 개 뿐이어서 1을 사용함

In [26]:
# 각각의 변수와 레이어는 다음과 같은 형태로 구성됩니다.
# W1 [3 3 1 32] -> [3 3]: 커널 크기, 1: 입력값 X 의 특성수, 32: 필터 갯수
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# tf.nn.conv2d 를 이용해 한칸씩 움직이는 컨볼루션 레이어를 쉽게 만들 수 있습니다.
# padding='SAME' 은 커널 슬라이딩시 최외곽에서 한칸 밖으로 더 움직이는 옵션
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

In [27]:
print(L1)

Tensor("Relu_4:0", shape=(?, 28, 28, 32), dtype=float32)


+ 3x3 컨볼루션

+ padding을 사용하면 이미지의 테두리까지도 조금 더 정확하게 평가 할 수 있음

+ padding="SAME" 옵션을 사용해서 전체 크기는 28x28로 유지됨

In [28]:
# Pooling 역시 tf.nn.max_pool 을 이용하여 쉽게 구성할 수 있습니다.
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# L1 = tf.nn.dropout(L1, keep_prob)

In [29]:
print(L1)

Tensor("MaxPool_4:0", shape=(?, 14, 14, 32), dtype=float32)


+ 커널 크기를 2x2로 하는 풀링

+ 스트라이드는 두 칸씩 움직임

+ 스트라이드가 2인 풀링이기 때문에 28/2=14, 크기가 14x14로 변함

In [30]:
# W2 의 [3, 3, 32, 64] 에서 32 는 L1 에서 출력된 W1 의 마지막 차원, 필터의 크기 입니다.
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

In [31]:
print(L2)

Tensor("Relu_5:0", shape=(?, 14, 14, 64), dtype=float32)


In [32]:
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# L2 = tf.nn.dropout(L2, keep_prob)

In [33]:
print(L2)

Tensor("MaxPool_5:0", shape=(?, 7, 7, 64), dtype=float32)


In [34]:
# FC 레이어: 입력값 7x7x64 -> 출력값 256
# Full connect를 위해 직전의 Pool 사이즈인 (?, 7, 7, 64) 를 참고하여 차원을 줄여줍니다.
#    Reshape  ->(?, 256)
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64]) # Fully connected layer로 만들어줌
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

In [35]:
# 최종 출력값 L3 에서의 출력 256개를 입력값으로 받아서 0~9 레이블인 10개의 출력값을 만듭니다.
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [36]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
# 최적화 함수를 RMSPropOptimizer 로 바꿔서 결과를 확인해봅시다.
# optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



## 신경망 모델 학습

코드 실행 엄청 오래걸림

In [37]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys,
                                          keep_prob: 0.7})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.355
Epoch: 0002 Avg. cost = 0.106
Epoch: 0003 Avg. cost = 0.080
Epoch: 0004 Avg. cost = 0.061
Epoch: 0005 Avg. cost = 0.050
Epoch: 0006 Avg. cost = 0.044
Epoch: 0007 Avg. cost = 0.035
Epoch: 0008 Avg. cost = 0.033
Epoch: 0009 Avg. cost = 0.029
Epoch: 0010 Avg. cost = 0.026
Epoch: 0011 Avg. cost = 0.022
Epoch: 0012 Avg. cost = 0.020
Epoch: 0013 Avg. cost = 0.018
Epoch: 0014 Avg. cost = 0.017
Epoch: 0015 Avg. cost = 0.016
최적화 완료!
